In [218]:
import pandas as pd
import numpy as np

YearJoined = pd.read_csv('./Ch02/data/year_joined.csv')
emails = pd.read_csv('./Ch02/data/emails.csv')
donations = pd.read_csv('./Ch02/data/donations.csv')

In [219]:
YearJoined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user        1000 non-null   int64 
 1   userStats   1000 non-null   object
 2   yearJoined  1000 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 23.6+ KB


In [220]:
# user status를 모두 1개씩 갖고 있다
YearJoined.groupby('user').count().groupby('userStats').count()
YearJoined.user.unique().shape

(1000,)

In [221]:
# week는 특정 순간이 아니라, 한 주를 모두 포함하는 정보이다
# week의 날짜는 일요일~토요일, 월요일~일요일 (1월1일 이런건 수정해야 한다)
emails.head()

,emailsOpened,user,week
0,3.0,1.0,2015-06-29 00:00:00
1,2.0,1.0,2015-07-13 00:00:00
2,2.0,1.0,2015-07-20 00:00:00
3,3.0,1.0,2015-07-27 00:00:00
4,1.0,1.0,2015-08-03 00:00:00


In [222]:
# 이메일을 열람한 경우만 데이터에 기재한 듯 싶다
emails[emails.emailsOpened < 1]

,emailsOpened,user,week


In [223]:
# 검토
emails.week = pd.to_datetime(emails.week)

for user in emails.user.unique():
    last = max(emails.loc[emails['user'] == user, 'week'])
    start = min(emails.loc[emails['user'] == user, 'week'])
    real_time = (last - start).days / 7
    my_data = emails[emails.user == user].shape[0]
    if real_time == my_data:
        print('이메일 열람하지 않은 경우는 제외한 데이터인게 확실함')
        break

이메일 열람하지 않은 경우는 제외한 데이터인게 확실함


In [224]:
# week와 user로 만들 수 있는 모든 경우의 수를 idx로 삼음
complete_idx = pd.MultiIndex.from_product((set(emails.week), set(emails.user)))
# len(set(emails.week)) == len(emails.week)

# 누락값을 채워서 인덱스로 만들기
all_email = emails.set_index(['week', 'user']).reindex(complete_idx, fill_value=0)
all_email = all_email.reset_index()
all_email.rename(columns={'level_0':'week', 'level_1':'member'}, inplace=True)

In [225]:
# 회원가입 이전이어서 0이 많다
all_email[all_email.member==319].sort_values('week').head()

,week,member,emailsOpened
15262,2015-02-09,319.0,0.0
75091,2015-02-16,319.0,0.0
7716,2015-02-23,319.0,0.0
35744,2015-03-02,319.0,0.0
80481,2015-03-09,319.0,0.0


In [288]:
# 가입일, 최종 수신일
cutoff_dates = emails.groupby('user').week.agg(['min', 'max']).reset_index()

# reset_index를 써야 for문에서 idx를 뽑아낸다
for _, row in cutoff_dates.reset_index().iterrows():
    print(row)
    break


for _, row in cutoff_dates.reset_index().iterrows():
    user = row['user']
    start_date = row['min']
    end_date = row['max']
    
    # 가입 이전 데이터 삭제
    start_idx = all_email.loc[all_email.member == user]
    start_idx = start_idx.loc[all_email.week < start_date].index
    all_email.drop(start_idx, inplace=True)
    
    # 탈퇴 이후 데이터 삭제
    end_idx = all_email.loc[all_email.member == user]
    end_idx = end_idx.loc[all_email.week > end_date].index
    all_email.drop(end_idx, inplace=True)

index                      0
user                     1.0
min      2015-06-29 00:00:00
max      2018-05-28 00:00:00
Name: 0, dtype: object


In [287]:
user_998 = all_email[all_email.member == 998]
user_998_uni = user_998.emailsOpened.unique()

max_998 = user_998.loc[user_998['emailsOpened'] == user_998_uni[2]].week.max()
min_998 = user_998.loc[user_998['emailsOpened'] == user_998_uni[1]].week.min()

print(min_998, max_998)

2017-12-04 00:00:00 2018-05-28 00:00:00


In [292]:
all_email[all_email.member == 998].week.min(), all_email[all_email.member == 998].week.max()

(Timestamp('2017-12-04 00:00:00'), Timestamp('2018-05-28 00:00:00'))